In [60]:
import pandas as pd 
import pyodbc
import numpy as np
import matplotlib as plt
from pandas import DataFrame

%matplotlib inline

#print(cursor.description)
#cursor.execute("DELETE FROM Smjestaj.Rezervacija WHERE ID=10")
#conn.commit()
#cursor.execute("SELECT * FROM Smjestaj.Rezervacija")
#for row in cursor:
#    print(row)


<h3>1. Indeksi i ogranicenja</h3><br>
    Baza sadrzi indeks za pretragu restorana u Splitu (filtrirajuci), i UNIQUE indeks na MjestoAktivnost (ista aktivnost moze samo jednom biti u istom mjestu). Dodano je i ogranicenje na Gost i Vlasnik da nemogu biti uneseni ako je Ime i Prezime isto

In [91]:
# Spoji se na bazu
conn = pyodbc.connect('Driver={SQL Server Native Client 11.0};'
                      'Server=DESKTOP-JBH3HM3\APARTMANI;'
                      'Database=Apartmani;'
                      'Trusted_Connection=yes;'
                      'UID=App_Uloga;'
                      'PWD=Apartman123')
cursor = conn.cursor()

# Prvi indeks
sql = "SELECT * FROM Destinacija.Restoran WHERE IDMjesto=1"
data = pd.read_sql(sql,conn)
print("--> Dohvati sve restorane u Splitu")
display(data)


# Drugi indeks
print("--> Pokusaj dodati istu aktivnost u isti grad (Split->Windsurf)\n")
try:
    cursor.execute("INSERT INTO Destinacija.MjestoAktivnost (IDMjesto, IDAktivnost) VALUES ('1', '2')")
    conn.commit()
except pyodbc.Error as err:
    print(err)
    print("\n")
    
print("--> Dohvati sve Aktivnosti")
sql = ("SELECT * FROM Destinacija.MjestoAktivnost"
       " INNER JOIN Destinacija.Mjesto ON MjestoAktivnost.IDMjesto=Mjesto.ID"
       " INNER JOIN Destinacija.Aktivnost ON MjestoAktivnost.IDAktivnost=Aktivnost.ID")
data = pd.read_sql(sql,conn)
display(data)


# Ogranicenje na Gost
print("--> Pokusaj dodati gosta s istim imenom i prezimenom (OIB drugaciji)\n")
try:
    cursor.execute("INSERT INTO Osoba.Gost (Ime, Prezime, OIB) VALUES ('Matko','Matic','44442444444')")
    conn.commit()
except pyodbc.Error as err:
    print(err)
    print("\n")

cursor.close()
conn.close()

--> Dohavti sve restorane u Splitu


,ID,Naziv,IDMjesto
0,1,Luna,1
1,5,Dina,1


--> Pokusaj dodati istu aktivnost u isti grad (Split->Windsurf)

('23000', "[23000] [Microsoft][SQL Server Native Client 11.0][SQL Server]Cannot insert duplicate key row in object 'Destinacija.MjestoAktivnost' with unique index 'IX_JednaAktivnostMjesto'. The duplicate key value is (2, 1). (2601) (SQLExecDirectW); [23000] [Microsoft][SQL Server Native Client 11.0][SQL Server]The statement has been terminated. (3621)")


--> Dohavti sve Aktivnosti


,ID,IDMjesto,IDAktivnost,ID,Naziv,PostanskiBroj,ID,Cijena,Naziv
0,11,1,1,1,Split,21000,1,250,Jet Ski
1,16,3,1,3,Zadar,23000,1,250,Jet Ski
2,12,1,2,1,Split,21000,2,100,WindSurf
3,17,3,2,3,Zadar,23000,2,100,WindSurf
4,13,2,3,2,Zagreb,10000,3,850,Skakanje padobranom
5,14,2,4,2,Zagreb,10000,4,100,Lunapark
6,15,2,5,2,Zagreb,10000,5,40,Kino
7,18,4,5,4,Rijeka,51000,5,40,Kino


--> Pokusaj dodati gosta s istim imenom i prezimenom (OIB drugaciji)

('23000', "[23000] [Microsoft][SQL Server Native Client 11.0][SQL Server]Violation of UNIQUE KEY constraint 'IstoImePrezimeGost'. Cannot insert duplicate key in object 'Osoba.Gost'. The duplicate key value is (Matko, Matic). (2627) (SQLExecDirectW); [23000] [Microsoft][SQL Server Native Client 11.0][SQL Server]The statement has been terminated. (3621)")




<h3>2. Uskladištene procedure i funkcije </h3><br>
    

In [107]:
# Spoji se na bazu
conn = pyodbc.connect('Driver={SQL Server Native Client 11.0};'
                      'Server=DESKTOP-JBH3HM3\APARTMANI;'
                      'Database=Apartmani;'
                      'Trusted_Connection=yes;'
                      'UID=App_Uloga;'
                      'PWD=Apartman123')
cursor = conn.cursor()

# Prva procedura
print("--> Dohvati samo dobro ocijenje apartmane")
sql = "EXEC usp_DobroOcjeneniApartmani"
data = pd.read_sql(sql,conn)
display(data)


# Druga procedura - 
print("\n--> Dohvati apartmane koji su blizu neke aktivnosti")
sql = "EXEC usp_ApartmaniBlizuAktivnosti 'Jet Ski'"
data = pd.read_sql(sql,conn)
display(data)


# Treca procedura
print("\n--> Unosimo rezervaciju u apartman koji je slobodan u tom periodu")
cursor.execute("EXEC usp_UnosRezervacije 1, 4, 3, '2020-11-02'")
conn.commit()   
               
sql = "SELECT * FROM Smjestaj.Rezervacija"     
data = pd.read_sql(sql,conn)
display(data)

print("\n--> Unosimo rezervaciju u apartman koji nije slobodan u tom periodu")
try:
    cursor.execute("EXEC usp_UnosRezervacije 1, 4, 3, '2020-11-02'")
    conn.commit()
except pyodbc.Error as err:
    print(err)
    print("\n")


# Cetvrta procedura - 
print("\n--> Dohvati koliko je dana zauzet apartman (od danas na dalje)")
cursor.execute("EXEC usp_ZauzetostApartmana 2")
for row in cursor:
    print("Apartman je zauzet " + str(row[0]) + " dan")



cursor.close()
conn.close()


--> Dohavti koliko je dana zauzet apartman (od danas na dalje)
Apartman je zauzet51dana


<h3>3. Okidaci </h3><br>
Prvi provjerava dali je korisnik koristio apartman prije nego sto ostavi ocjenu. Drugi i treci provjeravaju ispravnost OIBa
    

In [115]:
# Spoji se na bazu
conn = pyodbc.connect('Driver={SQL Server Native Client 11.0};'
                      'Server=DESKTOP-JBH3HM3\APARTMANI;'
                      'Database=Apartmani;'
                      'Trusted_Connection=yes;'
                      'UID=App_Uloga;'
                      'PWD=Apartman123')
cursor = conn.cursor()

# Prvi okidac
print("--> Pokusaj ostaviti recenziju od gost koji nije koristio apartman\n")
try:
    cursor.execute("INSERT INTO Smjestaj.Ocjena (Ocjena, IDApartman, IDGost, Komentar) VALUES ('1', '4', '2','Pre lose')")
    conn.commit()
except pyodbc.Error as err:
    print(err)
    print("\n")

# Drugi okidac
print("--> Pokusaj dodati gosta sa neispravnim OIBom\n")
try:
    cursor.execute("INSERT INTO Osoba.Gost (Ime, Prezime, OIB) VALUES ('Testko','Tests','1122112')")
    conn.commit()
except pyodbc.Error as err:
    print(err)
    print("\n")


    
cursor.close()
conn.close()

--> Pokusaj ostaviti recenziju od gost koji nije koristio apartman

('42000', '[42000] [Microsoft][SQL Server Native Client 11.0][SQL Server]Unos nije moguc jer korisnik nije koristio apartman (50000) (SQLExecDirectW)')


--> Pokusaj dodati gosta sa neispravnim OIBom

('42000', '[42000] [Microsoft][SQL Server Native Client 11.0][SQL Server]OIB mora sadrzavati tocno 11 brojeva (50000) (SQLExecDirectW)')




<h3>4. Sheme </h3><br>
Tablice su rasporedjene po sljedecim shemama:

Smjestaj
- apartman
- ocjena
- rezervacija

Destinacija
- mjesto
- aktivnost
- mjestoAktivnost
- restoran

Osobe
- gost
- vlasnik

<h1> PRESKOCIO SAM 5 6 7</h1>

  <h3>8. Kriptografija </h3><br>
  Broj kartice je dodan na gosta i vlasnika. Uz procedure za sifrirnje i desifriranje imamo i procedure za svaki unos koji poziva te procedure

In [162]:
# Spoji se na bazu
conn = pyodbc.connect('Driver={SQL Server Native Client 11.0};'
                      'Server=DESKTOP-JBH3HM3\APARTMANI;'
                      'Database=Apartmani;'
                      'Trusted_Connection=yes;'
                      'UID=App_Uloga;'
                      'PWD=Apartman123')
cursor = conn.cursor()
'''
# Prvi okidac
print("--> Pokusaj dodati karticu za gosta 1\n")
try:
    cursor.execute("EXEC usp_UnosKarticeGost 1, '2222-1111-4444-2222'")
    conn.commit()
except pyodbc.Error as err:
    print(err)
    print("\n")

print("--> Dohvati podatke gosta 1")
sql = "SELECT * FROM Osoba.Gost WHERE ID=1"
data = pd.read_sql(sql,conn)
display(data)

'''
# Sto god probam nemogu dohvatiti nazada broj kartice
# U SSMS normalno vrati tablicu tu ne radi
#print("--> Dohvati desifrirani broj kartice od gost 1")
#sql = "EXEC usp_BrojKarticeGost 1"
#data = pd.read_sql(sql,conn)
#display(data)


cursor.close()
conn.close()

In [163]:
# Spoji se na bazu
conn = pyodbc.connect('Driver={SQL Server Native Client 11.0};'
                      'Server=DESKTOP-JBH3HM3\APARTMANI;'
                      'Database=Apartmani;'
                      'Trusted_Connection=yes;'
                      'UID=App_Uloga;'
                      'PWD=Apartman123')
cursor = conn.cursor()
print("--> Dohvati podatke gostiju koristeci korisnika koji vidi maskirane podatke")
sql = "EXEC usp_DohvatiMaskiranePodatke"
data = pd.read_sql(sql,conn)
display(data)

cursor.close()
conn.close()

--> Dohvati podatke gosta 1


,ID,Ime,Prezime,OIB,BrojKartice,Email
0,1,Matko,Matic,44444444444,b'\x00\x98FE\xec\xd7\x9aG\xa7\xd8\xd2\xab\xd2P...,mXXX@XXXX.com
1,2,Ivan,Ivanic,55555555555,"b""\x00\x98FE\xec\xd7\x9aG\xa7\xd8\xd2\xab\xd2P...",None
2,3,Joza,Jozic,66666666666,None,None
3,4,Stef,Stefic,77777777777,None,None
